In [52]:
import os 
import codecs
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#Used to generate the same set of random numbers when random function is called
np.random.seed(7)

#Read The TSV Data File
data = pd.read_table('Datasets/train.tsv')
data1 = pd.read_table('Datasets/train_rel_2.tsv')

#Because of small size of the dataset O merged both the files given for training
data = [data, data1]
data = pd.concat(data)


essay_text = data['EssayText']
essay_score = data['Score1']
essay_set = data['EssaySet']

c:\users\mehak\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()
c:\users\mehak\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  if sys.path[0] == '':


In [53]:
## 
# RAW DATA ENCODING
# Format the text samples and labels into tensors that can be fed into a neural network
##
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

sets = [0,1,2,3] # The 4 different classes of Scores

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids

for label in sets:
	label_id = len(labels_index)
	labels_index[label] = label_id
	for t in essay_text[data['Score1']==label]:
		texts.append(t)
		labels.append(label_id)


print('Found %s texts.' % len(texts))

top_words = 5000 #top most-frequent words extracted from the dataset


tokenizer = Tokenizer(nb_words=top_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

max_response_length = 500
data = pad_sequences(sequences, maxlen=max_response_length)

#Convert class vector to binary class matrix, for use with categorical_crossentropy.
#Or in simple words to convert numbers into ONE-HOT Vector for MultiClass classification
labels = to_categorical(np.asarray(labels))

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


#Shuffle the dataset 
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


Found 34250 texts.
Found 16729 unique tokens.
Shape of data tensor: (34250, 500)
Shape of label tensor: (34250, 4)


In [19]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

#Set of stopwords from nltk.corpus package
stop = set(stopwords.words('english'))

word_counts_X = []  # List of word Counts in each example response
for x in essay_text:
    word_counts_X.append(len([i for i in word_tokenize(x) if i not in stop]))    

In [20]:
sent_counts = []  # Set of sentence counts in each example response
for x in essay_text:
    sent_counts.append(len(sent_tokenize(x)))

In [30]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import cohen_kappa_score

In [22]:
import re, collections
avg_word_length= []
for x in essay_text:
    clean_essay = re.sub(r'\W', ' ', x)
    words = word_tokenize(clean_essay)
    avg_word_length.append(sum(len(word) for word in words) / len(words))
 

In [27]:
# Tokenize a sentence into words

def sentence_to_wordlist(raw_sentence):
    
    clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)
    
    return tokens

In [28]:
import nltk
def tokenize(essay):
    stripped_essay = essay.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokenized_sentences.append(sentence_to_wordlist(raw_sentence))
    
    return tokenized_sentences

In [44]:
count_lemmas = []
for x in essay_text:
    tokenized_sentences = tokenize(x)      
    
    lemmas = []
    wordnet_lemmatizer = WordNetLemmatizer()
    
    for sentence in tokenized_sentences:
        tagged_tokens = nltk.pos_tag(sentence) 
        
        for token_tuple in tagged_tokens:
        
            pos_tag = token_tuple[1]
        
            if pos_tag.startswith('N'): 
                pos = wordnet.NOUN
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('J'):
                pos = wordnet.ADJ
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('V'):
                pos = wordnet.VERB
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('R'):
                pos = wordnet.ADV
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            else:
                pos = wordnet.NOUN
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
    
    lemma_count = len(set(lemmas))
    count_lemmas.append(lemma_count)
    

In [51]:
count_spell_error = []
data_file = open('Datasets/big.txt').read()
words_ = re.findall('[a-z]+', data_file.lower())
word_dict = collections.defaultdict(lambda: 0)
for word in words_:
    word_dict[word] += 1
for x in essay_text:

    
    clean_essay = re.sub(r'\W', ' ', str(x).lower())
    clean_essay = re.sub(r'[0-9]', '', clean_essay)
    
                        
    mispell_count = 0
    
    words = clean_essay.split()
                        
    for word in words:
        if not word in word_dict:
            mispell_count += 1
    
    count_spell_error.append(mispell_count)

In [36]:
count_noun=[]
count_adj=[]
count_verb =[]
count_adv=[]
for x in essay_text:
    tokenized_sentences = tokenize(x)
    
    noun_count = 0
    adj_count = 0
    verb_count = 0
    adv_count = 0
    
    for sentence in tokenized_sentences:
        tagged_tokens = nltk.pos_tag(sentence)
        
        for token_tuple in tagged_tokens:
            pos_tag = token_tuple[1]
        
            if pos_tag.startswith('N'): 
                noun_count += 1
            elif pos_tag.startswith('J'):
                adj_count += 1
            elif pos_tag.startswith('V'):
                verb_count += 1
            elif pos_tag.startswith('R'):
                adv_count += 1
            
    count_noun.append(noun_count)
    count_adj.append(adj_count)
    count_verb.append(verb_count)
    count_adv.append(adv_count)
    

In [38]:
#Shuffle all of the data
import numpy as np

word_counts_X = np.array(word_counts_X)
np.save("word_counts_X",word_counts_X)


sent_counts = np.array(sent_counts)
np.save("sent_counts",sent_counts)


avg_word_len = np.array(avg_word_length)
np.save("avg_word_len",avg_word_len)


count_noun = np.array(count_noun)
np.save("count_noun",count_noun)


count_adj = np.array(count_adj)
np.save("count_adj",count_adj)

count_verb = np.array(count_verb)
np.save("count_verb",count_verb)

count_adv = np.array(count_adv)
np.save("count_adv",count_adv)

count_lemmas = np.array(count_lemmas)
np.save("count_lemmas",count_lemmas)

count_spell_error = np.array(count_spell_error)
np.save("count_spell_error",count_spell_error)

essay_set = np.array(essay_set)
essay_set = essay_set[indices]

In [54]:
import numpy as np

word_counts_X = np.load("word_counts_X.npy")
sent_counts = np.load("sent_counts.npy")
avg_word_len = np.load("avg_word_len.npy")
count_noun = np.load("count_noun.npy")
count_adj  = np.load("count_adj.npy")
count_verb = np.load("count_verb.npy")
count_lemmas = np.load("count_lemmas.npy")
count_adv = np.load("count_adv.npy")
count_spell_error = np.load("count_spell_error.npy")

In [55]:
word_counts_X = word_counts_X[indices]
sent_counts = sent_counts[indices]
avg_word_len  = avg_word_len[indices]
count_noun  = count_noun[indices]
count_adj  = count_adj[indices]
count_verb  = count_verb[indices]
count_adv  = count_adv[indices]
count_lemmas  = count_lemmas[indices]
count_spell_error = count_spell_error[indices]


In [56]:
#To check whether the dataset is balanced or not
from collections import Counter
Counter(essay_set)


Counter({1: 3344,
         2: 2556,
         3: 3699,
         4: 3395,
         5: 3590,
         6: 3594,
         7: 3598,
         8: 3598,
         9: 3596,
         10: 3280})

In [73]:
#Size for train-test split
train_size = int(0.85 * len(data))

X_train = data[:train_size]
X_test = data[train_size:]

set_train = essay_set[:train_size]
set_test = essay_set[train_size:]

avg_word_len_train = avg_word_len[:train_size]
avg_word_len_test = avg_word_len[train_size:]

count_noun_train = count_noun[:train_size]
count_noun_test = count_noun[train_size:]

count_adj_train = count_adj[:train_size]
count_adj_test = count_adj[train_size:]

count_verb_train = count_verb[:train_size]
count_verb_test = count_verb[train_size:]

count_adv_train = count_adv[:train_size]
count_adv_test = count_adv[train_size:]

count_spell_error_train = count_spell_error[:train_size]
count_spell_error_test = count_spell_error[train_size:]

count_lemmas_train = count_lemmas[:train_size]
count_lemmas_test = count_lemmas[train_size:]


sent_count_train = sent_counts[:train_size]
sent_count_test = sent_counts[train_size:]

word_count_test = word_counts_X[train_size:]
word_count_train = word_counts_X[:train_size]

#Concatenate the three data-vectors into a single matrix or feature-set
features_train = np.column_stack((set_train,sent_count_train,word_count_train,avg_word_len_train,count_noun_train,count_adj_train,count_verb_train,count_adv_train,count_spell_error_train,count_lemmas_train))
features_test = np.column_stack((set_test,sent_count_test,word_count_test,avg_word_len_test,count_noun_test,count_adj_test,count_verb_test,count_adv_test,count_spell_error_test,count_lemmas_test))
#features_train = np.column_stack((set_train,sent_count_train,word_count_train))
#features_test = np.column_stack((set_test,sent_count_test,word_count_test))

y_train = labels[:train_size]
y_test = labels[train_size:]

print(X_train.shape)
print(features_train.shape)

(29112, 500)
(29112, 10)


In [74]:
#Model Design
#Functional Model

from keras.layers import Dense, Dropout, Activation, Input
from keras.layers import LSTM,Concatenate
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU
from keras.regularizers import l2

#Embedding-Output Vector Length
embedding_vector_length = 32

#Define Text-Input
text_in = Input(shape=(500,), name='text')

#Embeddings
#Used to convert the encoded data i.e the matrix of indices into a form compatible with LSTM
embedding = Embedding(output_dim=embedding_vector_length, input_dim=top_words, input_length=500)(text_in)

#LSTM
lstm_1= LSTM(100, return_sequences = True)(embedding)
lstm_2 = LSTM(150)(lstm_1)

#Features Inputs (essay_set, word_counts, sent_counts)
features_in = Input(shape=(10,), name='features')

#Merge layer to merge the output of LSTM and the feature inputs
x =  Concatenate()([lstm_2, features_in])

#Dropout for the hidden_units to be independent from each other
dropout = Dropout(0.2)(x)

#Hidden Dense Layer
D1 = Dense(200,W_regularizer=l2(0.01),b_regularizer = l2(0.01))(dropout) # try 150 dense after this
ED1 = ELU()(D1)

# Final Dense Output-Layer
score = Dense(4, activation='softmax', name='score',W_regularizer=l2(0.01),b_regularizer = l2(0.01))(ED1) 

#model
model = Model(input=[text_in, features_in], output=[score])

#optimizer
adam = Adam(lr = 0.001)#, decay = 1e-4) 
print(model.summary())

c:\users\mehak\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(200, kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
c:\users\mehak\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(4, activation="softmax", name="score", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
c:\users\mehak\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, 500)          0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 500, 32)      160000      text[0][0]                       
__________________________________________________________________________________________________
lstm_17 (LSTM)                  (None, 500, 100)     53200       embedding_9[0][0]                
__________________________________________________________________________________________________
lstm_18 (LSTM)                  (None, 150)          150600      lstm_17[0][0]                    
__________________________________________________________________________________________________
features (

In [ ]:
#configure the learning process
model.compile(optimizer=adam, loss='categorical_crossentropy',metrics=['accuracy'])

#Start Training
model.fit([X_train,features_train], y_train, nb_epoch = 1, batch_size=128, validation_split=0.1)  # we pass one data array per model input

c:\users\mehak\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 26200 samples, validate on 2912 samples
Epoch 1/1
 1152/26200 [>.............................] - ETA: 35:06 - loss: 4.7654 - acc: 0.3429

In [27]:
#Save The Trained Model 
model.save('functional_model.h5')

#Predict Score For the test set for calculating accuracy
scores = model.evaluate([X_test,features_test], y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.61%
